In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
from gensim.models import Word2Vec

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
fake_data = pd.read_csv('data/Fake.csv')
fake_data['label'] = 0
fake_data.info()

In [ ]:
true_data = pd.read_csv('data/True.csv')
true_data['label'] = 1
true_data.info()

In [ ]:
fake_data.isnull().sum()

In [ ]:
true_data.isnull().sum()

In [ ]:
fake_data['subject'].value_counts()

In [ ]:
true_data['subject'].value_counts()

In [ ]:
merged_data = pd.concat((fake_data, true_data))

In [ ]:
merged_data = merged_data.drop('date', axis=1)

In [ ]:
merged_data['text'] = merged_data['subject'] + ' ' + merged_data['title'] + ' ' + merged_data['text']
merged_data = merged_data.drop('subject', axis=1).drop('title', axis=1)

In [ ]:
random_permutation = np.random.permutation(len(merged_data))
merged_data = merged_data.iloc[random_permutation]

In [ ]:
merged_data = merged_data.head(1000)

In [ ]:
merged_data.info()

In [ ]:
merged_data['label'].value_counts()

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
def preprocess_text(text):
    ret_text = text.translate(text.maketrans({x:'' for x in string.punctuation})).lower() # lowercase; remove punctuation
    ret_text = nltk.tokenize.word_tokenize(ret_text)
    ret_text = [word for word in ret_text if word not in stopwords]
    ret_text = [lemmatizer.lemmatize(word) for word in ret_text]
    return  ret_text
    

In [ ]:
merged_data['text'] = merged_data['text'].apply(preprocess_text)

In [ ]:
w2v_text = Word2Vec(merged_data['text'], vector_size=100, workers=8)

In [ ]:
def vectorize(words, model):
    words_vecs = [model.wv[word] for word in words if word in model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

In [ ]:
merged_data['text'] = merged_data['text'].apply(lambda x: vectorize(x, w2v_text))

In [ ]:
merged_data['text'].head()

In [ ]:
merged_data.head()